In [73]:
import numpy as np
import random
from typing import Literal

In [74]:
#Types
Tile = str
Coordinates = tuple[int, int]
Up = tuple[Literal[1], Literal[0]]
Down = tuple[Literal[-1], Literal[0]]
Left = tuple[Literal[0], Literal[-1]]
Right = tuple[Literal[0], Literal[1]]
Direction = Up | Down | Left | Right
Compatibility = tuple[Tile, Tile, Direction]
Weights = dict[Tile, int]

UP = (-1, 0)
DOWN = (1, 0)
LEFT = (0, -1)
RIGHT = (0, 1)
DIRS = [UP, DOWN, LEFT, RIGHT]

# What we will do in this notebook ?
- Parse an input data to create rulesets for tiles
- Apply WFC to create a simple tiled version of WFC

In [75]:
class Cell:

    def __init__(self, pos, options):
        self.position = pos # position is given as a (y, x) tuple
        self.options = options # list of possible options available
        self.collapsed = False # collapsed state of the cell
        self.value = None # chosen option for this cell

    def __repr__(self):
        return f'tile {self.position} \n' \
               f'options : {[i for i in self.options]} \n' \
               f'value : {self.value} \n' \
               f'collapsed : {self.collapsed} \n'

    def enthropy(self):
        pass

    def collapse(self):
        pass

    def updateOptions(self):
        pass

    def __le__(self, other):
        return self.enthropy() < other.enthropy()

    def __eq__(self, other):
        return self.enthropy() == other.enthropy()

    def __gt__(self, other):
        return self.enthropy() > other.enthropy()


# Parsing
- Looking at every possible combinations in a NxN grid
- Returns a list of possible rules

Rules are represented by a three element tuple : (elem1, elem2, direction)
Each rule should come with its counterpart from the other side

In [76]:
input_matrix = np.array([
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','L','L','L'],
    ['L','C','C','L'],
    ['C','S','S','C'],
    ['S','S','S','S'],
    ['S','S','S','S'],
], dtype=Tile)

In [77]:
def valid_dirs(cur_coords:Coordinates, matrix_size:tuple[int, int]) -> list[Direction]:
    """
    Returns the valid directions from `cur_co_ord` in a matrix
    of `matrix_size`. Ensures that we don't try to take step to the
    left when we are already on the left edge of the matrix.
    :param cur_coords: the current coords
    :param matrix_size: the size of the matrix
    :return: a list of possible directions
    """
    y, x = cur_coords
    dirs : list[Coordinates] = []
    height, width = matrix_size

    if y > 0: dirs.append(UP)
    if y < height - 1 : dirs.append(DOWN)
    if x > 0: dirs.append(LEFT)
    if x < width - 1 : dirs.append(RIGHT)

    return dirs


In [88]:
def parseExemple_Matrix(matrix: np.ndarray):
    """
    Parses an example matrix. It extracts :
    1. Tile compatibilities
    2. The weight of each tile type

    :param matrix: a 2D matrix of tiles
    :return: A tuple :
        * A set of compatible tile combinations
        * A dict of weights for each tile type
    """
    compatibilities : set[Compatibility] = set()

    for y, row in enumerate(matrix):
        for x, elem in enumerate(row):
            #print(y, x, valid_dirs((y, x), matrix.shape))
            for d in valid_dirs((y, x), matrix.shape):
                other = matrix[y+d[0], x+d[1]]
                compatibilities.add((elem, other, d)) # Tile1, Tile2, Direction

    return compatibilities

In [89]:
parseExemple_Matrix(input_matrix)

{('C', 'C', (0, -1)),
 ('C', 'C', (0, 1)),
 ('C', 'L', (-1, 0)),
 ('C', 'L', (0, -1)),
 ('C', 'L', (0, 1)),
 ('C', 'S', (0, -1)),
 ('C', 'S', (0, 1)),
 ('C', 'S', (1, 0)),
 ('L', 'C', (0, -1)),
 ('L', 'C', (0, 1)),
 ('L', 'C', (1, 0)),
 ('L', 'L', (-1, 0)),
 ('L', 'L', (0, -1)),
 ('L', 'L', (0, 1)),
 ('L', 'L', (1, 0)),
 ('S', 'C', (-1, 0)),
 ('S', 'C', (0, -1)),
 ('S', 'C', (0, 1)),
 ('S', 'S', (-1, 0)),
 ('S', 'S', (0, -1)),
 ('S', 'S', (0, 1)),
 ('S', 'S', (1, 0))}